In [1]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Amroy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Amroy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Amroy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df_train = pd.read_json(r"D:\git\Taxonomy_LLM_models\icecat_data_train.json")
df_train.head()

,Brand,BrandInfo.BrandLocalName,BrandInfo.BrandLogo,BrandInfo.BrandName,BrandLogo,BrandPartCode,BulletPoints,Category.CategoryID,Category.Name.Language,Category.Name.Value,...,ProductSeries.Language,ProductSeries.Value,BulletPoints.BulletPointsId,BulletPoints.Language,BulletPoints.Updated,BulletPoints.Values,VirtualCategory,SummaryDescription,pathlist_ids,pathlist_names
1072689,ASUS,,https://images.icecat.biz/img/brand/thumb/161_...,ASUS,https://images.icecat.biz/img/brand/thumb/161_...,K31CD-IT049T,[],153,EN,PCs/Workstations,...,None,None,NaN,None,None,None,"[{'VirtualCategoryID': 195, 'UNCATID': '431718...",NaN,2833>150>153,Computers & Electronics>Computers>PCs/Workstat...
906402,HP,,https://images.icecat.biz/img/brand/thumb/1_91...,HP,https://images.icecat.biz/img/brand/thumb/1_91...,686915-A41,[],2509,EN,Notebook Spare Parts,...,None,None,NaN,None,None,None,None,NaN,2833>150>8355>2509,Computers & Electronics>Computers>Notebook Par...
411281,C2G,,https://images.icecat.biz/img/brand/thumb/2834...,C2G,https://images.icecat.biz/img/brand/thumb/2834...,37745,[],953,EN,Fibre Optic Cables,...,None,None,NaN,None,None,None,None,NaN,2833>830>953,Computers & Electronics>Computer Cables>Fibre ...
425903,HP,,https://images.icecat.biz/img/brand/thumb/1_91...,HP,https://images.icecat.biz/img/brand/thumb/1_91...,FA889AA#AC3,[],8194,EN,Handheld Mobile Computer Spare Parts,...,None,None,NaN,None,None,None,None,NaN,2833>150>8194,Computers & Electronics>Computers>Handheld Mob...
1047582,Lenovo,,https://images.icecat.biz/img/brand/thumb/728_...,Lenovo,https://images.icecat.biz/img/brand/thumb/728_...,109559U,[],153,EN,PCs/Workstations,...,None,None,NaN,None,None,None,"[{'VirtualCategoryID': 194, 'UNCATID': '431718...",NaN,2833>150>153,Computers & Electronics>Computers>PCs/Workstat...


In [4]:
df_validate = pd.read_json(r"D:\git\Taxonomy_LLM_models\icecat_data_validate.json")
df_validate.head()

,Brand,BrandInfo.BrandLocalName,BrandInfo.BrandLogo,BrandInfo.BrandName,BrandLogo,BrandPartCode,BulletPoints,Category.CategoryID,Category.Name.Language,Category.Name.Value,...,ProductSeries.Language,ProductSeries.Value,BulletPoints.BulletPointsId,BulletPoints.Language,BulletPoints.Updated,BulletPoints.Values,VirtualCategory,SummaryDescription,pathlist_ids,pathlist_names
309544,Fujitsu,,https://images.icecat.biz/img/brand/thumb/15_b...,Fujitsu,https://images.icecat.biz/img/brand/thumb/15_b...,S26361-F5524-L800,[],1563,EN,Internal Solid State Drives,...,None,None,NaN,None,None,None,None,NaN,2833>206>2840>1563,Computers & Electronics>Data Storage>Data Stor...
805366,PanzerGlass,,https://images.icecat.biz/img/brand/thumb/1016...,PanzerGlass,https://images.icecat.biz/img/brand/thumb/1016...,PG1501,[],1568,EN,Screen Protectors,...,None,None,NaN,None,None,None,None,NaN,2833>107>1568,Computers & Electronics>Telecom & Navigation>S...
126809,2-Power,,https://images.icecat.biz/img/brand/thumb/1520...,2-Power,https://images.icecat.biz/img/brand/thumb/1520...,ALT268563B,[],911,EN,Memory Modules,...,None,None,NaN,None,None,None,None,NaN,2833>106>2844>911,Computers & Electronics>Computer Components>Sy...
922232,Lenovo,,https://images.icecat.biz/img/brand/thumb/728_...,Lenovo,https://images.icecat.biz/img/brand/thumb/728_...,25205062,[],2509,EN,Notebook Spare Parts,...,None,None,NaN,None,None,None,None,NaN,2833>150>8355>2509,Computers & Electronics>Computers>Notebook Par...
567207,Verbatim,,https://images.icecat.biz/img/brand/thumb/669_...,Verbatim,https://images.icecat.biz/img/brand/thumb/669_...,97537,[],194,EN,Keyboards,...,None,None,NaN,None,None,None,None,NaN,2833>191>194,Computers & Electronics>Data Input Devices>Key...


In [6]:
df_test = pd.read_json(r"D:\git\Taxonomy_LLM_models\icecat_data_test.json")
df_test.head()

,Brand,BrandInfo.BrandLocalName,BrandInfo.BrandLogo,BrandInfo.BrandName,BrandLogo,BrandPartCode,BulletPoints,Category.CategoryID,Category.Name.Language,Category.Name.Value,...,ProductSeries.Language,ProductSeries.Value,BulletPoints.BulletPointsId,BulletPoints.Language,BulletPoints.Updated,BulletPoints.Values,VirtualCategory,SummaryDescription,pathlist_ids,pathlist_names
500081,Fujitsu,,https://images.icecat.biz/img/brand/thumb/15_b...,Fujitsu,https://images.icecat.biz/img/brand/thumb/15_b...,FSP:G-SW3Z560PRE0S,[],788,EN,Warranty & Support Extensions,...,None,None,NaN,None,None,None,None,NaN,2833>839>788,Computers & Electronics>Warranty & Support>War...
741063,HP,,https://images.icecat.biz/img/brand/thumb/1_91...,HP,https://images.icecat.biz/img/brand/thumb/1_91...,5KM83PA,None,151,EN,Notebooks,...,EN,13,1268560.0,EN,2018-10-21 21:55:51,"[Windows 10 Home 64-bit, Intel® Core™ i7-8565U...","[{'VirtualCategoryID': 329, 'UNCATID': '432115...",NaN,2833>150>151,Computers & Electronics>Computers>Notebooks
1091454,HP,,https://images.icecat.biz/img/brand/thumb/1_91...,HP,https://images.icecat.biz/img/brand/thumb/1_91...,4NG33EA,[],153,EN,PCs/Workstations,...,EN,880,NaN,None,None,None,"[{'VirtualCategoryID': 195, 'UNCATID': '431718...",NaN,2833>150>153,Computers & Electronics>Computers>PCs/Workstat...
522928,C2G,,https://images.icecat.biz/img/brand/thumb/2834...,C2G,https://images.icecat.biz/img/brand/thumb/2834...,83061,[],883,EN,Networking Cables,...,None,None,NaN,None,None,None,None,NaN,2833>830>883,Computers & Electronics>Computer Cables>Networ...
479478,Lenovo,,https://images.icecat.biz/img/brand/thumb/728_...,Lenovo,https://images.icecat.biz/img/brand/thumb/728_...,5PS0A14091,[],788,EN,Warranty & Support Extensions,...,None,None,NaN,None,None,None,None,NaN,2833>839>788,Computers & Electronics>Warranty & Support>War...


In [7]:
df_train.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 489902 entries, 1072689 to 1008771
Data columns (total 45 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Brand                                       489902 non-null  object 
 1   BrandInfo.BrandLocalName                    489902 non-null  object 
 2   BrandInfo.BrandLogo                         488765 non-null  object 
 3   BrandInfo.BrandName                         489902 non-null  object 
 4   BrandLogo                                   488765 non-null  object 
 5   BrandPartCode                               489902 non-null  object 
 6   BulletPoints                                455119 non-null  object 
 7   Category.CategoryID                         489902 non-null  int64  
 8   Category.Name.Language                      489902 non-null  object 
 9   Category.Name.Value                         489902 non-null  ob

In [8]:
df_train.isna().sum() #check for null values

Brand                                              0
BrandInfo.BrandLocalName                           0
BrandInfo.BrandLogo                             1137
BrandInfo.BrandName                                0
BrandLogo                                       1137
BrandPartCode                                      0
BulletPoints                                   34783
Category.CategoryID                                0
Category.Name.Language                             0
Category.Name.Value                                0
Description.Disclaimer                         53728
Description.ID                                 53728
Description.LeafletPDFURL                      53728
Description.LongDesc                           53728
Description.LongProductName                    53728
Description.ManualPDFSize                      61104
Description.ManualPDFURL                       53728
Description.MiddleDesc                         53728
Description.PDFSize                           

In [9]:
# Show all columns
pd.set_option('display.max_columns', None)
print(df_train.head()) 

          Brand BrandInfo.BrandLocalName  \
1072689    ASUS                            
906402       HP                            
411281      C2G                            
425903       HP                            
1047582  Lenovo                            

                                       BrandInfo.BrandLogo  \
1072689  https://images.icecat.biz/img/brand/thumb/161_...   
906402   https://images.icecat.biz/img/brand/thumb/1_91...   
411281   https://images.icecat.biz/img/brand/thumb/2834...   
425903   https://images.icecat.biz/img/brand/thumb/1_91...   
1047582  https://images.icecat.biz/img/brand/thumb/728_...   

        BrandInfo.BrandName  \
1072689                ASUS   
906402                   HP   
411281                  C2G   
425903                   HP   
1047582              Lenovo   

                                                 BrandLogo BrandPartCode  \
1072689  https://images.icecat.biz/img/brand/thumb/161_...  K31CD-IT049T   
906402   https://images

In [10]:
df_train=df_train[["Title","Description.LongDesc","SummaryDescription.LongSummaryDescription","SummaryDescription.ShortSummaryDescription","Description.LongProductName","pathlist_names"]]
df_train

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names
1072689,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,<b>Smart Multimedia Performance</b><br>\nVivoP...,ASUS K31CD-IT049T. Processor frequency: 3.4 GH...,"ASUS K31CD-IT049T, 3.4 GHz, 6th gen Intel® Cor...","Intel Core i7-6700 (8M Cache, 3.4GHz), 16GB RA...",Computers & Electronics>Computers>PCs/Workstat...
906402,HP 686915-A41 notebook spare part Keyboard,,HP 686915-A41. Type: Keyboard. Keyboard langua...,"HP 686915-A41, Keyboard, Belgian, Keyboard bac...",Keyboard in midnight black finish with backlig...,Computers & Electronics>Computers>Notebook Par...
411281,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,Get the performance you demand at a price that...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,1m ST/SC Plenum-Rated 9/125 Duplex Single-Mode...,Computers & Electronics>Computer Cables>Fibre ...
425903,HP FA889AA Battery,Keeping an extra source of power nearby means ...,"HP FA889AA. Product type: Battery, Product col...","HP FA889AA, Battery, White, Lithium-Ion (Li-Io...","1100 mAh, Lithium Ion, Standard Battery",Computers & Electronics>Computers>Handheld Mob...
1047582,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,The C30 builds on its award-winning design as ...,Lenovo ThinkStation C30. Processor frequency: ...,"Lenovo ThinkStation C30, 2 GHz, Intel® Xeon® E...","Intel Xeon E5-2620 (15M Cache, 2.00 GHz, 7.20 ...",Computers & Electronics>Computers>PCs/Workstat...
...,...,...,...,...,...,...
1093316,Fujitsu ESPRIMO P758 8th gen Intel® Core™ i7 i...,The FUJITSU ESPRIMO P758 Desktop provides high...,Fujitsu ESPRIMO P758. Processor frequency: 3.2...,"Fujitsu ESPRIMO P758, 3.2 GHz, 8th gen Intel® ...","Intel Core i7-8700, 1 x 16 GB DDR4 2666 MHz, P...",Computers & Electronics>Computers>PCs/Workstat...
886679,Acer 55.PDS07.001 notebook spare part Power board,,"Acer 55.PDS07.001. Type: Power board, Brand co...","Acer 55.PDS07.001, Power board, Acer",Power board,Computers & Electronics>Computers>Notebook Par...
35003,Gigabyte Aire M1 mouse USB Optical 1000 DPI,<b>Mini Size</b><br>\nUltimate Choice for Note...,Gigabyte Aire M1. Movement detection technolog...,"Gigabyte Aire M1, Optical, USB, 1000 DPI, Black","Optical, USB, 1000dpi, 61g, Black",Computers & Electronics>Data Input Devices>Mice
779133,HP Pavilion x360 14-dh0062no Gold Hybrid (2-in...,None,HP Pavilion x360 14-dh0062no. Product type: Hy...,"HP Pavilion x360 14-dh0062no, 8th gen Intel® C...",None,Computers & Electronics>Computers>Notebooks


In [11]:
df_train.isna().sum()

Title                                             0
Description.LongDesc                          53728
SummaryDescription.LongSummaryDescription         0
SummaryDescription.ShortSummaryDescription        0
Description.LongProductName                   53728
pathlist_names                                    0
dtype: int64

In [12]:
# Fill NaN with empty strings
df_train.fillna("", inplace=True)

In [13]:
df_train.isna().sum()

Title                                         0
Description.LongDesc                          0
SummaryDescription.LongSummaryDescription     0
SummaryDescription.ShortSummaryDescription    0
Description.LongProductName                   0
pathlist_names                                0
dtype: int64

In [ ]:
# Combine 6 textual columns into one
def combine_text(row):
    return " ".join([
        row["Title"],
        row["Description.LongDesc"],
        row["SummaryDescription.LongSummaryDescription"],
        row["SummaryDescription.ShortSummaryDescription"],
        row["Description.LongProductName"]
        #row["pathlist_names"]
    ])

In [15]:
df_train['raw_text'] = df_train.apply(combine_text,axis=1)

In [16]:
df_train.head(2)

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text
1072689,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,<b>Smart Multimedia Performance</b><br>\nVivoP...,ASUS K31CD-IT049T. Processor frequency: 3.4 GH...,"ASUS K31CD-IT049T, 3.4 GHz, 6th gen Intel® Cor...","Intel Core i7-6700 (8M Cache, 3.4GHz), 16GB RA...",Computers & Electronics>Computers>PCs/Workstat...,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...
906402,HP 686915-A41 notebook spare part Keyboard,,HP 686915-A41. Type: Keyboard. Keyboard langua...,"HP 686915-A41, Keyboard, Belgian, Keyboard bac...",Keyboard in midnight black finish with backlig...,Computers & Electronics>Computers>Notebook Par...,HP 686915-A41 notebook spare part Keyboard HP...


In [17]:
# Initialize NLTK tools
stop = set(stopwords.words("english"))
lemma = WordNetLemmatizer()
punc = set(punctuation)

In [18]:
# Cleaning function
def clean_text(text):
    tokens = word_tokenize(text.lower())
    word_tokens = [t for t in tokens if t.isalpha()]
    clean_tokens = [t for t in word_tokens if t not in stop]
    lemmatized_tokens = [lemma.lemmatize(t, pos="n") for t in clean_tokens]
    return " ".join(lemmatized_tokens)

In [20]:
from tqdm import tqdm
tqdm.pandas()
df_train["cleaned_text"] = df_train["raw_text"].progress_apply(clean_text)

100%|██████████| 489902/489902 [13:25<00:00, 608.33it/s]


In [23]:
df_train[['raw_text','cleaned_text']]

,raw_text,cleaned_text
1072689,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,asus pc gen gb gb hdd black tower b smart mult...
906402,HP 686915-A41 notebook spare part Keyboard HP...,hp notebook spare part keyboard hp type keyboa...
411281,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,duplex fiber patch cable blue fiber optic cabl...
425903,HP FA889AA Battery Keeping an extra source of ...,hp battery keeping extra source power nearby m...
1047582,Lenovo ThinkStation C30 Intel® Xeon® E5 Family...,lenovo thinkstation family gb gb hdd black tow...
...,...,...
1093316,Fujitsu ESPRIMO P758 8th gen Intel® Core™ i7 i...,fujitsu esprimo gen gb gb ssd black red midi t...
886679,Acer 55.PDS07.001 notebook spare part Power bo...,acer notebook spare part power board acer type...
35003,Gigabyte Aire M1 mouse USB Optical 1000 DPI <b...,gigabyte aire mouse usb optical dpi b mini siz...
779133,HP Pavilion x360 14-dh0062no Gold Hybrid (2-in...,hp pavilion gold hybrid cm x pixel touchscreen...


In [21]:
df_train.to_csv("icecat_preprocessed_train.csv", index=False)

In [25]:
df_train.to_json("icecat_preprocessed_train.json")

In [26]:
df_train.head(3)

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text,cleaned_text
1072689,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,<b>Smart Multimedia Performance</b><br>\nVivoP...,ASUS K31CD-IT049T. Processor frequency: 3.4 GH...,"ASUS K31CD-IT049T, 3.4 GHz, 6th gen Intel® Cor...","Intel Core i7-6700 (8M Cache, 3.4GHz), 16GB RA...",Computers & Electronics>Computers>PCs/Workstat...,ASUS K31CD-IT049T PC 6th gen Intel® Core™ i7 i...,asus pc gen gb gb hdd black tower b smart mult...
906402,HP 686915-A41 notebook spare part Keyboard,,HP 686915-A41. Type: Keyboard. Keyboard langua...,"HP 686915-A41, Keyboard, Belgian, Keyboard bac...",Keyboard in midnight black finish with backlig...,Computers & Electronics>Computers>Notebook Par...,HP 686915-A41 notebook spare part Keyboard HP...,hp notebook spare part keyboard hp type keyboa...
411281,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,Get the performance you demand at a price that...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,1m ST/SC Plenum-Rated 9/125 Duplex Single-Mode...,Computers & Electronics>Computer Cables>Fibre ...,C2G 1m ST/SC Plenum-Rated 9/125 Duplex Single-...,duplex fiber patch cable blue fiber optic cabl...


In [27]:
# Applying same technique for test and validation data

In [28]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# Use the same columns as training
use_cols = [
    "Title",
    "Description.LongDesc",
    "SummaryDescription.LongSummaryDescription",
    "SummaryDescription.ShortSummaryDescription",
    "Description.LongProductName",
    "pathlist_names"
]

In [29]:
# Load validation and test sets
df_validate = pd.read_json(r"D:\git\Taxonomy_LLM_models\icecat_data_validate.json")[use_cols]
df_test = pd.read_json(r"D:\git\Taxonomy_LLM_models\icecat_data_test.json")[use_cols]


In [30]:
df_validate.head(2)

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names
309544,Fujitsu S26361-F5524-L800 internal solid state...,Fujitsu offers a full line of Hard Drives and ...,Fujitsu S26361-F5524-L800. SSD capacity: 800 G...,"Fujitsu S26361-F5524-L800, 800 GB, 2.5"", 6 Gbit/s","800GB Read-Inten 6.35 cm (2.5"") , SATA III, Ho...",Computers & Electronics>Data Storage>Data Stor...
805366,PanzerGlass PG1501 screen protector Clear scre...,,PanzerGlass PG1501. Type: Clear screen protect...,"PanzerGlass PG1501, Clear screen protector, Mo...",Protective Anti Scratch Fluid Resistant Glass ...,Computers & Electronics>Telecom & Navigation>S...


In [31]:
df_test.head(2)

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names
500081,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,In times of growing complexity and decreasing ...,Fujitsu FSP:G-SW3Z560PRE0S. Number of years: 3...,"Fujitsu FSP:G-SW3Z560PRE0S, 3 year(s), 9x5","SP 3y TS Sub & Upgr, 9x5, 4h Rm Rt f/ CS200c A...",Computers & Electronics>Warranty & Support>War...
741063,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",<b>Revolutionary battery life on a convertible...,HP Spectre x360 13-ap0023tu. Product type: Hyb...,"HP Spectre x360 13-ap0023tu, 8th gen Intel® Co...","Intel® Core™ i7-8565U (1.8 GHz), 16GB DDR4-SDR...",Computers & Electronics>Computers>Notebooks


In [36]:
# Fill NaNs with empty strings
df_validate.fillna("", inplace=True)
df_test.fillna("", inplace=True)

In [37]:
df_validate["raw_text"] = df_validate.apply(combine_text, axis=1)
df_test["raw_text"] = df_test.apply(combine_text, axis=1)

In [38]:
df_validate.head(2)

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text
309544,Fujitsu S26361-F5524-L800 internal solid state...,Fujitsu offers a full line of Hard Drives and ...,Fujitsu S26361-F5524-L800. SSD capacity: 800 G...,"Fujitsu S26361-F5524-L800, 800 GB, 2.5"", 6 Gbit/s","800GB Read-Inten 6.35 cm (2.5"") , SATA III, Ho...",Computers & Electronics>Data Storage>Data Stor...,Fujitsu S26361-F5524-L800 internal solid state...
805366,PanzerGlass PG1501 screen protector Clear scre...,,PanzerGlass PG1501. Type: Clear screen protect...,"PanzerGlass PG1501, Clear screen protector, Mo...",Protective Anti Scratch Fluid Resistant Glass ...,Computers & Electronics>Telecom & Navigation>S...,PanzerGlass PG1501 screen protector Clear scre...


In [39]:
df_test.head(2)

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text
500081,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,In times of growing complexity and decreasing ...,Fujitsu FSP:G-SW3Z560PRE0S. Number of years: 3...,"Fujitsu FSP:G-SW3Z560PRE0S, 3 year(s), 9x5","SP 3y TS Sub & Upgr, 9x5, 4h Rm Rt f/ CS200c A...",Computers & Electronics>Warranty & Support>War...,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...
741063,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",<b>Revolutionary battery life on a convertible...,HP Spectre x360 13-ap0023tu. Product type: Hyb...,"HP Spectre x360 13-ap0023tu, 8th gen Intel® Co...","Intel® Core™ i7-8565U (1.8 GHz), 16GB DDR4-SDR...",Computers & Electronics>Computers>Notebooks,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid..."


In [40]:
# Clean validation and test sets
df_validate["cleaned_text"] = df_validate["raw_text"].progress_apply(clean_text)
df_test["cleaned_text"] = df_test["raw_text"].progress_apply(clean_text)


100%|██████████| 153095/153095 [03:39<00:00, 696.69it/s]


In [41]:
df_validate.head()

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text,cleaned_text
309544,Fujitsu S26361-F5524-L800 internal solid state...,Fujitsu offers a full line of Hard Drives and ...,Fujitsu S26361-F5524-L800. SSD capacity: 800 G...,"Fujitsu S26361-F5524-L800, 800 GB, 2.5"", 6 Gbit/s","800GB Read-Inten 6.35 cm (2.5"") , SATA III, Ho...",Computers & Electronics>Data Storage>Data Stor...,Fujitsu S26361-F5524-L800 internal solid state...,fujitsu internal solid state drive gb serial a...
805366,PanzerGlass PG1501 screen protector Clear scre...,,PanzerGlass PG1501. Type: Clear screen protect...,"PanzerGlass PG1501, Clear screen protector, Mo...",Protective Anti Scratch Fluid Resistant Glass ...,Computers & Electronics>Telecom & Navigation>S...,PanzerGlass PG1501 screen protector Clear scre...,panzerglass screen protector clear screen prot...
126809,2-Power ALT268563B memory module 8 GB DDR4 266...,<b>2-POWER MEMORY</b><br>\n2-Power memory modu...,"2-Power ALT268563B. Component for: PC/server, ...","2-Power ALT268563B, 8 GB, 1 x 8 GB, DDR4, 2666...",8GB DDR4 2666MHz Replaces 2P-L09284-850,Computers & Electronics>Computer Components>Sy...,2-Power ALT268563B memory module 8 GB DDR4 266...,memory module gb mhz b memory br memory module...
922232,Lenovo 25205062 notebook spare part Keyboard,- High quality<br>\n- Long life<br>\n- Easy in...,Lenovo 25205062. Type: Keyboard. Keyboard lang...,"Lenovo 25205062, Keyboard, Dutch, Lenovo, Idea...",Keyboard for Ideapad S300,Computers & Electronics>Computers>Notebook Par...,Lenovo 25205062 notebook spare part Keyboard -...,lenovo notebook spare part keyboard high quali...
567207,Verbatim 97537 keyboard Bluetooth QWERTY Black,"On the go, looking for a quick and easy way to...",Verbatim 97537. Connectivity technology: Wirel...,"Verbatim 97537, Wireless, Bluetooth, QWERTY, B...","Wireless, Bluetooth, iOS 4, f/iPad/iPhone/iPod...",Computers & Electronics>Data Input Devices>Key...,Verbatim 97537 keyboard Bluetooth QWERTY Black...,verbatim keyboard bluetooth qwerty black go lo...


In [42]:
df_test.head()

,Title,Description.LongDesc,SummaryDescription.LongSummaryDescription,SummaryDescription.ShortSummaryDescription,Description.LongProductName,pathlist_names,raw_text,cleaned_text
500081,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,In times of growing complexity and decreasing ...,Fujitsu FSP:G-SW3Z560PRE0S. Number of years: 3...,"Fujitsu FSP:G-SW3Z560PRE0S, 3 year(s), 9x5","SP 3y TS Sub & Upgr, 9x5, 4h Rm Rt f/ CS200c A...",Computers & Electronics>Warranty & Support>War...,Fujitsu FSP:G-SW3Z560PRE0S warranty/support ex...,fujitsu fsp extension time growing complexity ...
741063,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",<b>Revolutionary battery life on a convertible...,HP Spectre x360 13-ap0023tu. Product type: Hyb...,"HP Spectre x360 13-ap0023tu, 8th gen Intel® Co...","Intel® Core™ i7-8565U (1.8 GHz), 16GB DDR4-SDR...",Computers & Electronics>Computers>Notebooks,"HP Spectre x360 13-ap0023tu Blue,Silver Hybrid...",hp spectre blue silver hybrid cm x pixel touch...
1091454,HP OMEN 880-156nf 8th gen Intel® Core™ i7 i7-8...,,HP OMEN 880-156nf. Processor frequency: 3.2 GH...,"HP OMEN 880-156nf, 3.2 GHz, 8th gen Intel® Cor...",,Computers & Electronics>Computers>PCs/Workstat...,HP OMEN 880-156nf 8th gen Intel® Core™ i7 i7-8...,hp omen gen gb gb black tower pc hp omen proce...
522928,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,Perfect for your home office or a large instal...,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,Cat5E Assembled UTP Patch Cable Green 1m,Computers & Electronics>Computer Cables>Networ...,C2G 1m Cat5e Non-Booted Unshielded (UTP) Netwo...,unshielded utp network patch cable green perfe...
479478,Lenovo 5PS0A14091 warranty/support extension,Lenovo offers a comprehensive portfolio of val...,"Lenovo 5PS0A14091. Number of years: 3 year(s),...","Lenovo 5PS0A14091, 3 year(s), On-site, 24x7, N...",3YR Onsite + Keep Your Drive,Computers & Electronics>Warranty & Support>War...,Lenovo 5PS0A14091 warranty/support extension L...,lenovo extension lenovo offer comprehensive po...


In [43]:
# Save results to JSON
df_validate.to_json("icecat_preprocessed_validate.json", orient="records", lines=False)
df_test.to_json("icecat_preprocessed_test.json", orient="records", lines=False)